<h1> Parameter Tunning 
    
    Look at different steps & batch sizes 
    (6 months of data used)

# Importing libraries:

In [30]:
!pip install pandas
!pip install sklearn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# Import data; fill nan values; split data intro train and test:

In [31]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# for later use
features_num = 15

# 2018 data
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values
data.fillna(method = 'ffill', inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

# Apply feature scaling:

In [32]:
from sklearn.preprocessing import MinMaxScaler

# data scaling
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Importing the Keras libraries and packages:

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []
hist_list = []
y_pred_list = []
prediction_list = []
time_count = []

# Prepare of data according to LSTM needs,  create regressor & tune:

In [34]:
# parameters
# steps = 96
# n_hidden = 1 - NO HIDDEN LAYERS HERE
units = 100
# batch_size = 96

from sklearn.model_selection import ParameterGrid

# 48 - day; 96 - 2 days; 336 - 1 week
parameters = {'steps': [48, 96, 336],
              'batch_size': [48, 96, 336]}

all_param = ParameterGrid(parameters)

# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

for i in range(len(all_param)):
    
    start_time = time.time()
    
    # set parameters right
    steps = all_param[i]['steps']
    batch_size = all_param[i]['batch_size']
    
    # divide features and labels
    X_train = data_train[:, 0:15] 
    y_train = data_train[:, -1]
    X_test = data_test[:, 0:15] 
    y_test = data_test[:, -1] 

    # divide data into train and test 
    X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)
    
    # put data into correct shape
    X_train, y_train = split_data(X_train, y_train, steps)
    X_test, y_test = split_data(X_test, y_test, steps)
    X_val, y_val = split_data(X_val, y_val, steps)

    X_train = cut_data(X_train, batch_size)
    y_train = cut_data(y_train, batch_size)
    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)
    X_val = cut_data(X_val, batch_size)
    y_val = cut_data(y_val, batch_size)
    
    # design the LSTM
    def regressor_tunning(kernel_initializer = 'he_normal',
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='linear'))
        model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = 'Adamax')
        return model

    model = regressor_tunning()

    #apply patience callback
    #early_stopping = EarlyStopping(monitor = 'mse', patience= 50)

    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = 180,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    # reset states to have suitable predictions
    model.reset_states()
    
    hist_list.append(history.history['mse'])
    hist_list.append(history.history['val_mse'])
    hist_list.append(history.history['mae'])
    hist_list.append(history.history['val_mae'])

    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)

    y_pred_list.append(y_pred)
    
    # prices col = 15
    y_pred = (y_pred * sc_X.data_range_[15]) + (sc_X.data_min_[15])
    y_test = (y_test * sc_X.data_range_[15]) + (sc_X.data_min_[15])

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525

    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================

    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)


    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)

    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)

    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)

    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)


Epoch 1/180
87/87 [==============================] - 7s 77ms/step - loss: 0.2274 - mse: 0.2274 - mae: 0.3609 - val_loss: 0.0250 - val_mse: 0.0250 - val_mae: 0.1419
Epoch 2/180
87/87 [==============================] - 5s 53ms/step - loss: 0.1326 - mse: 0.1326 - mae: 0.2898 - val_loss: 0.0202 - val_mse: 0.0202 - val_mae: 0.1282
Epoch 3/180
87/87 [==============================] - 4s 51ms/step - loss: 0.1094 - mse: 0.1094 - mae: 0.2624 - val_loss: 0.0087 - val_mse: 0.0087 - val_mae: 0.0813
Epoch 4/180
87/87 [==============================] - 6s 67ms/step - loss: 0.0921 - mse: 0.0921 - mae: 0.2418 - val_loss: 0.0179 - val_mse: 0.0179 - val_mae: 0.1230
Epoch 5/180
87/87 [==============================] - 5s 59ms/step - loss: 0.0803 - mse: 0.0803 - mae: 0.2252 - val_loss: 0.0074 - val_mse: 0.0074 - val_mae: 0.0740
Epoch 6/180
87/87 [==============================] - 5s 56ms/step - loss: 0.0620 - mse: 0.0620 - mae: 0.1992 - val_loss: 0.0048 - val_mse: 0.0048 - val_mae: 0.0578
Epoch 7/180
87/8

87/87 [==============================] - 4s 48ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0184 - val_loss: 4.9528e-04 - val_mse: 4.9528e-04 - val_mae: 0.0179
Epoch 97/180
87/87 [==============================] - 4s 46ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0184 - val_loss: 6.1131e-04 - val_mse: 6.1131e-04 - val_mae: 0.0212
Epoch 98/180
87/87 [==============================] - 4s 52ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0185 - val_loss: 4.5802e-04 - val_mse: 4.5802e-04 - val_mae: 0.0164
Epoch 99/180
87/87 [==============================] - 5s 53ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0182 - val_loss: 5.8924e-04 - val_mse: 5.8924e-04 - val_mae: 0.0206
Epoch 100/180
87/87 [==============================] - 4s 47ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0182 - val_loss: 4.6135e-04 - val_mse: 4.6135e-04 - val_mae: 0.0166
Epoch 101/180
87/87 [==============================] - 4s 49ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0176 - val_loss: 4.7403e-04 - val_mse: 4.

87/87 [==============================] - 4s 48ms/step - loss: 9.6280e-04 - mse: 9.6280e-04 - mae: 0.0165 - val_loss: 4.5659e-04 - val_mse: 4.5659e-04 - val_mae: 0.0121
Epoch 143/180
87/87 [==============================] - 4s 48ms/step - loss: 9.5920e-04 - mse: 9.5920e-04 - mae: 0.0165 - val_loss: 4.4385e-04 - val_mse: 4.4385e-04 - val_mae: 0.0130
Epoch 144/180
87/87 [==============================] - 4s 51ms/step - loss: 9.5647e-04 - mse: 9.5647e-04 - mae: 0.0164 - val_loss: 4.5150e-04 - val_mse: 4.5150e-04 - val_mae: 0.0121
Epoch 145/180
87/87 [==============================] - 4s 46ms/step - loss: 9.5118e-04 - mse: 9.5118e-04 - mae: 0.0164 - val_loss: 4.5504e-04 - val_mse: 4.5504e-04 - val_mae: 0.0125
Epoch 146/180
87/87 [==============================] - 4s 47ms/step - loss: 9.4732e-04 - mse: 9.4732e-04 - mae: 0.0163 - val_loss: 4.4446e-04 - val_mse: 4.4446e-04 - val_mae: 0.0121
Epoch 147/180
87/87 [==============================] - 4s 47ms/step - loss: 9.3900e-04 - mse: 9.3900e-04

Epoch 8/180
86/86 [==============================] - 9s 106ms/step - loss: 0.0165 - mse: 0.0165 - mae: 0.1021 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0266
Epoch 9/180
86/86 [==============================] - 9s 104ms/step - loss: 0.0136 - mse: 0.0136 - mae: 0.0910 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0296
Epoch 10/180
86/86 [==============================] - 9s 104ms/step - loss: 0.0112 - mse: 0.0112 - mae: 0.0825 - val_loss: 9.9488e-04 - val_mse: 9.9488e-04 - val_mae: 0.0230
Epoch 11/180
86/86 [==============================] - 9s 100ms/step - loss: 0.0094 - mse: 0.0094 - mae: 0.0761 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0356
Epoch 12/180
86/86 [==============================] - 9s 100ms/step - loss: 0.0080 - mse: 0.0080 - mae: 0.0695 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0253
Epoch 13/180
86/86 [==============================] - 9s 103ms/step - loss: 0.0065 - mse: 0.0065 - mae: 0.0623 - val_loss: 9.8792e-04 - val_mse: 9.8792e-04 - val_ma

Epoch 102/180
86/86 [==============================] - 9s 102ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0178 - val_loss: 4.9981e-04 - val_mse: 4.9981e-04 - val_mae: 0.0177
Epoch 103/180
86/86 [==============================] - 9s 101ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0178 - val_loss: 5.9866e-04 - val_mse: 5.9866e-04 - val_mae: 0.0206
Epoch 104/180
86/86 [==============================] - 8s 96ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0174 - val_loss: 4.9013e-04 - val_mse: 4.9013e-04 - val_mae: 0.0173
Epoch 105/180
86/86 [==============================] - 8s 94ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0175 - val_loss: 5.5251e-04 - val_mse: 5.5251e-04 - val_mae: 0.0192
Epoch 106/180
86/86 [==============================] - 8s 92ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0175 - val_loss: 4.6555e-04 - val_mse: 4.6555e-04 - val_mae: 0.0162
Epoch 107/180
86/86 [==============================] - 8s 90ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0175 - val_loss: 5.474

...

12/12 [==============================] - 4s 317ms/step - loss: 0.0107 - mse: 0.0107 - mae: 0.0818 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0249
Epoch 63/180
12/12 [==============================] - 4s 333ms/step - loss: 0.0103 - mse: 0.0103 - mae: 0.0798 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0251
Epoch 64/180
12/12 [==============================] - 4s 324ms/step - loss: 0.0097 - mse: 0.0097 - mae: 0.0771 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0251
Epoch 65/180
12/12 [==============================] - 4s 335ms/step - loss: 0.0099 - mse: 0.0099 - mae: 0.0771 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0266
Epoch 66/180
12/12 [==============================] - 4s 333ms/step - loss: 0.0094 - mse: 0.0094 - mae: 0.0752 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0253
Epoch 67/180
12/12 [==============================] - 5s 392ms/step - loss: 0.0097 - mse: 0.0097 - mae: 0.0767 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0259
Epoch 68/180
12/1

12/12 [==============================] - 4s 367ms/step - loss: 0.0038 - mse: 0.0038 - mae: 0.0458 - val_loss: 7.5855e-04 - val_mse: 7.5855e-04 - val_mae: 0.0215
Epoch 111/180
12/12 [==============================] - 4s 345ms/step - loss: 0.0037 - mse: 0.0037 - mae: 0.0452 - val_loss: 8.1180e-04 - val_mse: 8.1180e-04 - val_mae: 0.0224
Epoch 112/180
12/12 [==============================] - 4s 362ms/step - loss: 0.0038 - mse: 0.0038 - mae: 0.0461 - val_loss: 7.7642e-04 - val_mse: 7.7642e-04 - val_mae: 0.0217
Epoch 113/180
12/12 [==============================] - 4s 375ms/step - loss: 0.0038 - mse: 0.0038 - mae: 0.0463 - val_loss: 8.3886e-04 - val_mse: 8.3886e-04 - val_mae: 0.0228
Epoch 114/180
12/12 [==============================] - 4s 344ms/step - loss: 0.0037 - mse: 0.0037 - mae: 0.0447 - val_loss: 8.2580e-04 - val_mse: 8.2580e-04 - val_mae: 0.0226
Epoch 115/180
12/12 [==============================] - 4s 343ms/step - loss: 0.0036 - mse: 0.0036 - mae: 0.0440 - val_loss: 8.0579e-04 - va

Epoch 157/180
12/12 [==============================] - 4s 322ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0325 - val_loss: 6.6064e-04 - val_mse: 6.6064e-04 - val_mae: 0.0193
Epoch 158/180
12/12 [==============================] - 4s 332ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0336 - val_loss: 6.8743e-04 - val_mse: 6.8743e-04 - val_mae: 0.0199
Epoch 159/180
12/12 [==============================] - 4s 320ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0331 - val_loss: 6.8448e-04 - val_mse: 6.8448e-04 - val_mae: 0.0197
Epoch 160/180
12/12 [==============================] - 4s 325ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0324 - val_loss: 6.3706e-04 - val_mse: 6.3706e-04 - val_mae: 0.0185
Epoch 161/180
12/12 [==============================] - 4s 322ms/step - loss: 0.0021 - mse: 0.0021 - mae: 0.0319 - val_loss: 6.3894e-04 - val_mse: 6.3894e-04 - val_mae: 0.0188
Epoch 162/180
12/12 [==============================] - 4s 324ms/step - loss: 0.0021 - mse: 0.0021 - mae: 0.0321 - val_loss: 6

11/11 [==============================] - 19s 2s/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0414 - val_loss: 6.0692e-04 - val_mse: 6.0692e-04 - val_mae: 0.0176
Epoch 74/180
11/11 [==============================] - 19s 2s/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0407 - val_loss: 6.0400e-04 - val_mse: 6.0400e-04 - val_mae: 0.0175
Epoch 75/180
11/11 [==============================] - 20s 2s/step - loss: 0.0030 - mse: 0.0030 - mae: 0.0397 - val_loss: 6.0311e-04 - val_mse: 6.0311e-04 - val_mae: 0.0175
Epoch 76/180
11/11 [==============================] - 19s 2s/step - loss: 0.0030 - mse: 0.0030 - mae: 0.0398 - val_loss: 5.9703e-04 - val_mse: 5.9703e-04 - val_mae: 0.0174
Epoch 77/180
11/11 [==============================] - 19s 2s/step - loss: 0.0029 - mse: 0.0029 - mae: 0.0392 - val_loss: 6.1524e-04 - val_mse: 6.1524e-04 - val_mae: 0.0171
Epoch 78/180
11/11 [==============================] - 19s 2s/step - loss: 0.0029 - mse: 0.0029 - mae: 0.0384 - val_loss: 5.9914e-04 - val_mse: 5.9914e-04

11/11 [==============================] - 24s 2s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0217 - val_loss: 4.8273e-04 - val_mse: 4.8273e-04 - val_mae: 0.0152
Epoch 168/180
11/11 [==============================] - 24s 2s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0220 - val_loss: 4.8066e-04 - val_mse: 4.8066e-04 - val_mae: 0.0154
Epoch 169/180
11/11 [==============================] - 24s 2s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0217 - val_loss: 4.9701e-04 - val_mse: 4.9701e-04 - val_mae: 0.0163
Epoch 170/180
11/11 [==============================] - 23s 2s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0216 - val_loss: 4.8412e-04 - val_mse: 4.8412e-04 - val_mae: 0.0153
Epoch 171/180
11/11 [==============================] - 24s 2s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0216 - val_loss: 4.6659e-04 - val_mse: 4.6659e-04 - val_mae: 0.0139
Epoch 172/180
11/11 [==============================] - 23s 2s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0217 - val_loss: 4.7306e-04 - val_mse: 4.730

# Save results into a data frame

In [36]:
# Save
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
                       
                        'time': time_count})


In [37]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal,time
0,37.936108,24.495981,29.808339,19.809879,38.981722,25.180467,825.453125
1,31.552693,20.439026,24.990323,16.307403,32.450644,21.078962,1580.458508
2,33.449770,20.715460,32.140521,19.587816,33.674863,20.913865,7064.752007
3,32.604840,23.086320,29.421052,19.963627,33.070565,23.569986,889.551930
4,30.926347,18.035266,24.851826,14.822518,31.763481,18.532880,2053.163445
5,33.251305,23.188203,33.568608,20.976006,33.192212,23.597869,8391.761726
6,36.464584,29.372006,33.595820,27.253947,36.904590,29.712212,458.254601
7,39.774452,31.487159,34.951449,28.111644,40.495625,32.029341,809.491827
8,30.630364,22.402153,21.520397,18.336396,32.175782,23.232792,4319.335325


In [41]:
# Save
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
                       
                        'time': time_count}, index = all_param)

results.to_csv('Results_LSTM_steps_batch.csv')
results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal,time
"{'batch_size': 48, 'steps': 48}",37.936108,24.495981,29.808339,19.809879,38.981722,25.180467,825.453125
"{'batch_size': 48, 'steps': 96}",31.552693,20.439026,24.990323,16.307403,32.450644,21.078962,1580.458508
"{'batch_size': 48, 'steps': 336}",33.449770,20.715460,32.140521,19.587816,33.674863,20.913865,7064.752007
"{'batch_size': 96, 'steps': 48}",32.604840,23.086320,29.421052,19.963627,33.070565,23.569986,889.551930
"{'batch_size': 96, 'steps': 96}",30.926347,18.035266,24.851826,14.822518,31.763481,18.532880,2053.163445
"{'batch_size': 96, 'steps': 336}",33.251305,23.188203,33.568608,20.976006,33.192212,23.597869,8391.761726
"{'batch_size': 336, 'steps': 48}",36.464584,29.372006,33.595820,27.253947,36.904590,29.712212,458.254601
"{'batch_size': 336, 'steps': 96}",39.774452,31.487159,34.951449,28.111644,40.495625,32.029341,809.491827
"{'batch_size': 336, 'steps': 336}",30.630364,22.402153,21.520397,18.336396,32.175782,23.232792,4319.335325
